# 📌 Documentação Resumida - Camada Ouro

## 📖 Importação de Bibliotecas
Carrega funções do PySpark para **agregação e ordenação de dados** e importa `DeltaTable` para **operações de merge** no Delta Lake.

## 📖 Leitura dos Dados da Camada Prata
Carrega os dados transformados da **Camada Prata** para processamento e agregação.

## 📖 Criação e Atualização da Tabela Ouro
- **Verifica se a tabela ouro existe** no Delta Lake.
- Se **não existir**, cria a tabela.
- Se **já existir**, realiza um **merge incremental**, garantindo que **novos registros sejam inseridos** e **dados existentes sejam atualizados**.

## 📖 Agregação dos Dados
- **Agrupa os dados** por `nm_tipo_cervejaria` e `Localizacao`.
- Calcula a **quantidade de cervejarias por tipo e local**.
- Ordena os resultados para facilitar análises.

## 📖 Salvamento dos Dados na Camada Ouro
- Salva os **dados agregados** na Camada Ouro em formato **Delta Lake**.

## 📖 Exibição dos Dados
Exibe a tabela processada com **número total de cervejarias por tipo e localização**.


In [0]:
from pyspark.sql.functions import count, desc
from delta.tables import DeltaTable


In [0]:
%run "./Variaveis"

In [0]:
df = spark.read.format("delta").load(f"{camada_prata}cervejarias")
df.printSchema()

In [0]:
df_reduzido = df.select('id_cervejaria','nm_cervejaria','nm_tipo_cervejaria','Localizacao')


In [0]:
caminho_ouro = f"{camada_ouro}cervejarias"


if not DeltaTable.isDeltaTable(spark, caminho_ouro):
    print("criando a tabela ouro")
    df_reduzido.write.format("delta").mode("overwrite").save(caminho_ouro)

delta_destino = DeltaTable.forPath(spark, caminho_ouro)
delta_destino.optimize()

delta_destino.alias('destino').merge(
    df_reduzido.alias("novo"),
    "destino.id_cervejaria = novo.id_cervejaria"
).whenMatchedUpdate(set={
    "nm_cervejaria": "novo.nm_cervejaria",
    "nm_tipo_cervejaria": "novo.nm_tipo_cervejaria",
    "Localizacao": "novo.Localizacao",
}).whenNotMatchedInsert(values={
    "id_cervejaria": "novo.id_cervejaria",
    "nm_cervejaria": "novo.nm_cervejaria",
    "nm_tipo_cervejaria": "novo.nm_tipo_cervejaria",
    "Localizacao": "novo.Localizacao",
}).execute()


In [0]:
df_ouro = spark.read.format("delta").load(caminho_ouro)


In [0]:
df_ouro = df_ouro.groupBy("nm_tipo_cervejaria","Localizacao").agg(count('nm_cervejaria').alias('count')).orderBy('nm_tipo_cervejaria',desc('count'))
df_ouro = df_ouro.withColumnRenamed('count',"Total_Empresas_por_Localizacao")


In [0]:
df_ouro.write.format("delta").mode("overwrite").save(f"{camada_ouro}cervejarias_agregradas")

In [0]:
display(df_ouro)